In [ ]:
%matplotlib inline

import time
import math
import numpy as np
import socket
import struct
import pyaudio
import multiprocessing
import threading
import requests
from time import perf_counter

import matplotlib.pyplot as plt
import aubio

In [ ]:
LED_COUNT = 254
SPLIT_LED = 254

In [ ]:
p = pyaudio.PyAudio()

dev_count = p.get_device_count()
for dev_index in range(dev_count):
    print(p.get_device_info_by_index(dev_index)["name"])
mix_prefix = "VoiceMeeter Output"
#mix_prefix = "Stereo Mix"
#mix_prefix = "Speakers (Steam Streaming S"

In [ ]:
audio_sr = 44100
chunksize = 128
hopsize = chunksize // 2
beat_hist = 10
median_hist = 5
decay_val = 0.7

total_frames = 0
beat_count = 0
beat_times = []
bps_list = []
blip_val = 0.0
data_dec = np.zeros(hopsize)
window = np.blackman(hopsize)
wheel_pos = 0
bps_med = 0.0
pos_inv = False

def wheel(pos):
    """Generate rainbow colors across 0-255 positions."""
    if pos < 85:
        return np.array((pos * 3, 255 - pos * 3, 0))
    elif pos < 170:
        pos -= 85
        return np.array((255 - pos * 3, 0, pos * 3))
    else:
        pos -= 170
        return np.array((0, pos * 3, 255 - pos * 3))

def run_leds():
    global blip_val
    global wheel_pos
    global pos_inv
    led_state = np.zeros((SPLIT_LED, 3))
    fft_state = np.abs(np.fft.rfft(data_dec))
    while True:
        wheel_pos += 1
        wheel_pos = wheel_pos % 255
        data_fft = np.abs(np.fft.rfft(data_dec * window))
        fft_state = fft_state * decay_val + data_fft * (1.0 - decay_val) 
        for i in range(0, SPLIT_LED // 2):
            col_pos = int((i / (SPLIT_LED // 2)) * 255.0 + wheel_pos) % 255
            fft_pos = int((i / (SPLIT_LED // 2)) * len(data_fft) * 0.75)
            fft_norm = fft_state / np.max(fft_state)
            fft_val = max(0.01, min(math.pow(fft_norm[fft_pos], 0.5), 1.0))
            col = wheel(col_pos) / 255.0
        
            if pos_inv == True:
                blip_weight = math.pow(abs((SPLIT_LED // 2) - i) / (SPLIT_LED // 2), 3.0)
            else:
                blip_weight = math.pow(1.0 - abs((SPLIT_LED // 2) - i) / (SPLIT_LED // 2), 3.0)
            blip_weight = blip_weight * blip_val
            
            led_state[SPLIT_LED // 2 + i][0] = col[0] * fft_val + blip_weight
            led_state[SPLIT_LED // 2 + i][1] = col[1] * fft_val + blip_weight
            led_state[SPLIT_LED // 2 + i][2] = col[2] * fft_val + blip_weight
            led_state[SPLIT_LED // 2 - i][0] = col[0] * fft_val + blip_weight
            led_state[SPLIT_LED // 2 - i][1] = col[1] * fft_val + blip_weight
            led_state[SPLIT_LED // 2 - i][2] = col[2] * fft_val + blip_weight
        
        if not math.isnan(bps_med):
            bps_pos = int(((math.sin(bps_med * time.time()) + 1.0) / 2.0) * SPLIT_LED)
            for i in range(max(0, bps_pos - 3), min(SPLIT_LED, bps_pos + 3)):
                led_state[i][0] = 1.0 - led_state[i][0]
                led_state[i][1] = 1.0 - led_state[i][1]
                led_state[i][2] = 1.0 - led_state[i][2]
        
        updates = []
        for i, led_col in enumerate(led_state):
            updates.append(i)
            updates.append(min(int(led_state[i][0] * 200), 255))
            updates.append(min(int(led_state[i][1] * 200), 255))
            updates.append(min(int(led_state[i][2] * 200), 255))
        #updates = []
        #for i in range(0, 50):
        #    updates.append(int(i + 70))
        #    updates.append(int(blip_val * 200))
        #    updates.append(int(blip_val * 200))
        #    updates.append(int(blip_val * 200))
        updates = ",".join(list(map(str, updates)))
        requests.post("http://10.1.1.105:9000/set_many", params = {"updates": updates})
        blip_val *= 0.7

threading.Thread(target=run_leds).start()

In [ ]:
dev_count = p.get_device_count()
for dev_index in range(dev_count):
    if p.get_device_info_by_index(dev_index)["name"].startswith(mix_prefix):
        break

stream = p.open(
    format = pyaudio.paFloat32,
    channels = 1,
    rate = audio_sr,
    input = True,
    frames_per_buffer = hopsize,
    input_device_index = dev_index
)

beat_detector = aubio.tempo("default", chunksize, hopsize, audio_sr)


while(True):
    data = stream.read(hopsize)
    data_dec = np.array(np.frombuffer(data, dtype="float32")[:])
    total_frames += len(data_dec)    
    data_dec[data_dec < 0.0001] = 0.0
    #print(np.min(data_dec), np.max(data_dec))
    is_beat = beat_detector(data_dec)
    if is_beat:
        beat_count += 1
        beat_times.append(total_frames / audio_sr)
        bps = 0
        if len(beat_times) > beat_hist:
            bps = beat_hist / (beat_times[-1] - beat_times[-1 - beat_hist])
            bps_list.append(bps)
        bps_med = np.median(bps_list[-median_hist:])
        #print("beat ->", bps_med * 60)
        blip_val = 1.0
        if pos_inv == True:
            pos_inv = False
        else:
            pos_inv = True
    beat_times = beat_times[-100:]
    bps_list = bps_list[-100:]